In [ ]:
import os
from glob import glob

import nivapy3 as nivapy
import resa_uv_abs as uv

# Upload UV absorbance data to RESA2

This notebook reads UV absorbance data produced by NIVA Lab's spectrophotometer and adds it to RESA2. It provides an alternative to Tore's old Access database named `ABSDATA_IMPORT.accdb`. This is a temporary solution, but will hopefully be faster/more efficient than the current workflow.

#### Description of workflow

 * The analyses are conducted in batches by Erling Bratsberg. Result files for each batch are stored in folders named `AB{yymmdd}` where `{yymmdd}` is the day the analyses were conducted. Files are stored both on the Lab's network drive at `T:\LAMBDA40P\UVWINLAB\DATA` and by Liv Bente at `K:\Avdeling\412 Ana\LAMBDA\ABSSPEKTER_KAU`. The directory structure in these locations is broadly - but not exactly - the same. **Need to check which source is definitive** <br><br>
 
 * Within each folder, result files are named `xxxxx.SP` where `xxxxx` is the serial number used in Labware. Each sample can be uniquely identified in Labware by the Labware text ID, which is formed as `NR-{year}-{serial_no}`. Each file contains a header (**check what information this contains**), plus UV absorbance results for 701 wavelengths from 200 to 900 nm inclusive <br><br>
 
 * Each folder should also contain one or more "blank"/calibration files, usually named either `BLANK.SP` or `BL.SP`. Sometimes both files are present. In this case, I **need to find out how to choose the correct blank for each result file** <br><br>
 
 * The folders may contain files with other extensions, but these can be ignored for the purposes of this workflow <br><br>
 
 * The spectrophotometer uses cuvettes with a fixed length of 5 cm (**check this**). UV absorbance is usually reported per cm, so values should be divided by 5 before upload <br><br>
 
 * Sometimes, very dark samples must be diluted before analysis. In these cases, a dilution factor should also be reported. **Need to find out where this is stored and include it in the code** <br><br>

Corrected values are calculated as
 
$$A_{cor} = \frac{D(A_{raw} - A_{blank})}{L}$$

where $A_{cor}$ is the corrected value to be uploaded; $A_{raw}$ and $A_{blank}$ are the raw and calibration absorbances at each wavelength, respectively; $D$ is the dimensionless dilution factor; and $L$ is the cuvette length in cm.

## 1. Connect to Nivabasen

You must login as a user with sufficient permissions to write to tables in RESA2.

In [ ]:
engine = nivapy.da.connect()

## 2. User options

Take care with setting `force_update = True` in the code below. This will delete existing data from the database and upload it again.

#### Issues to solve

 * The correct blank file, cuvette length and dilution factor should eventually be identified automatically in the code. I believe the cuvette length is fixed at 5 cm, but am not sure where the other information is stored. Defaults are currently hard-coded for testing, but **this must be changed before the code is used "for real"** <br><br>
 
 * The data already in RESA2 currently uses one of the two methods in the table below. **Check which is correct for the current instrument** (Tore's Access database assumes method ID 10666, which has more complete metadata)
 
| wc_method_id |      name      |  unit  | laboratory | method_ref |        description       |
|:------------:|:--------------:|:------:|:----------:|:----------:|:------------------------:|
|     10666    | LAMBDA_Spekter | abs/cm |    NIVA    |            | Hele spekteret, filtrert |
|     10758    | Sensor_Spekter |        |            |            |                          |

In [ ]:
# Data folder to process
uv_data_fold = r"../../test_data"

# Blank file to use (usually either 'BLANK.SP' or 'BL.SP'
blank_file = "BLANK.SP"

# Cuvette length in cm
cuvette_len_cm = 5

# Dilution of sample (dimensionless)
dilution = 1

# Method ID to assign in RESA. Either 10666 or 10758.
# Not sure which is correct: 10758 is more recent,
# but 10666 is more complete
meth_id = 10666

# Whether to re-upload data for samples already in the database.
# If 'force_update' is False and UV data for the current water sample are
# already present in RESA2, data for this sample will be skipped (and a
# message printed to the output). If 'force_update' is True, data already
# in the database for the current sample will be deleted and the values
# uploaded again
force_update = False

In [ ]:
# Check user input
assert blank_file in (
    "BL.SP",
    "BLANK.SP",
), "'blank_file' must be either 'BL.SP' or 'BLANK.SP'"
assert meth_id in (10666, 10758), "'meth_id' must be either 10666 or 10758."
assert isinstance(cuvette_len_cm, (int, float)), "'cuvette_len_cm' must be a number."
assert isinstance(dilution, (int, float)), "'dilution' must be a number."
assert type(force_update) is bool, "'force_update' must be of Boolean type."

## 3. Upload data

The code performs the following steps:

 1. Gets a list of all folders within `uv_data_fold` that begin with the letters `AB` (capitalised) <br><br>
 
 2. For each folder, gets a list of all files ending with extension `.SP` <br><br>
 
 3. Reads the specified blank file from within the folder, and checks that it contains data for 701 wavelengths <br><br>
 
 4. For each (non-blank) `.SP` file in each folder, identifies the serial number from the file name. Also identifies the year when the analysis was conducted for both the blank and raw files (this is extracted from the file headers). Constructs the Labware text ID as `{NR-{year}-{serial_no}` and then uses this information to identify the corresponding RESA2 water sample ID. The code raises an error if more than one sample ID is found, and prints a warning (but continues) if no matching ID can be identified <br><br>
 
 5. Reads each raw data file, and checks they contain data for 701 wavelengths <br><br>
 
 6. The blank values are subtracted from the raw values and then corrected for dilution and cuvette length <br><br>
 
 7. Corrected values are uploaded to `RESA2.ABSORBANCE_SPECTRAS`. If this table already contains values for the water sample being processed, a warning will be printed and the code will continue to the next file without uploading anything (unless `force_update = True`, in which case the existing data will be deleted from RESA and uploaded again) <br><br>
 
 8. Within each folder, a subfolder is created named `uploaded`. Each raw `.SP` file that is successfully processed is moved to this subfolder <br><br>
 
 9. When a sample has been successfully processed, a new row is inserted into `RESA2.LOG_ABS_SPECTRA`. This records key information about the sample (`labware_text_id`, `water_sample_id`, `year`, `serial_no`, `blank_file`, `dilution`, `cuvette_len_cm`, `original_path` and `archive_path`), plus the user that uploaded the data and the current date. Note that if a file is uploaded multiple times, it will appear several times in this table (but the dataset can only appear once in `RESA2.ABSORBANCE_SPECTRAS`, as it must be deleted each time before uploading again)

In [ ]:
# Relevant folder names begin with "AB"
folders = glob(f"{uv_data_fold}/*")
folders = [fold for fold in folders if os.path.split(fold)[1][:2] == "AB"]

for fold in folders:
    flist = glob(f"{fold}/*.SP")

    if len(flist) > 0:
        print("##################################################################")
        print(fold)
        print("##################################################################")

        blank_path = f"{fold}/{blank_file}"
        blank_year = uv.get_year(blank_path)
        blank_df = uv.read_uv_abs(blank_path)

        for fpath in flist:
            serial_no = os.path.split(fpath)[1][:-3]

            if serial_no not in ("BL", "BLANK"):
                data_year = uv.get_year(fpath)
                assert (
                    data_year == blank_year
                ), "Years for 'blank' and 'raw' samples don't match."
                ws_id = uv.get_water_sample_id(serial_no, data_year, engine)

                if ws_id is None:
                    print(
                        f"Skipping upload for NR-{data_year}-{serial_no}. "
                        "Could not identify water sample in RESA2. "
                    )
                else:
                    df = uv.read_uv_abs(fpath)
                    df = uv.correct_values(
                        df, blank_df, cuvette_len_cm, dilution, ws_id, meth_id
                    )
                    uv.add_to_resa(
                        df,
                        fold,
                        data_year,
                        serial_no,
                        blank_file,
                        dilution,
                        cuvette_len_cm,
                        engine,
                        force_update=force_update,
                    )